In [2]:
import numpy as np
from EM import *

In [3]:
from scipy.optimize import fsolve

In [54]:
def OConnell_Budiansky_fl(K0,G0,Kfl,crd, alpha_):
    """ Saturated effective elastic moduli using the O’Connell and Budiansky Self Consistent (SC) formulations under the constraints of small aspect ratio cracks with soft-fluid saturation.

    Args:
        K0 (GPa): bulk modulus of background medium
        G0 (GPa): shear modulus of background medium
        Kfl (GPa): bulk modulus of soft fluid inclusion, e.g gas
        crd (unitless): crack density
        alpha_ (unitless): aspect ratio

    Returns:
        K_sat,G_sat: elastic moduli of cracked background fully saturated by soft fluid.

    Refs: 
        O’Connell and Budiansky, (1974)
    """    
    
    nu0 = (3*K0-2*G0)/(6*K0+2*G0)#  Poisson ratio of the uncracked solid
    w = Kfl/alpha_/K0
    # given crack density and w, solve for the D and nu_eff simulaneously using equations 23 and 25 in O’Connell and Budiansky, (1974)
    nu_eff, D =  fsolve(OC_R_funcs, (0.2, 0.9), args = (crd,nu0,w))   

    K_sat = K0*(1-16*(1-nu_eff**2)*crd*D/(9*(1-2*nu_eff)))
    G_sat = G0*(1-32/45*(1-nu_eff) *(D+ 3/(2-nu_eff))*crd)
    return K_sat,G_sat

def OC_R_funcs(params, crd,nu_0,w ): # crd, nu_0,w 
    """Form the system of equastions to solve. Given crack density and w, solve for the D and nu_eff simulaneously using equations 23 and 25 in O’Connell and Budiansky, (1974)

    Args:
        params (list): Parameters to solve, in the form of [nu_eff, D]
        crd (unitless): crack density
        nu_0 (): Poisson's ratio of background medium
        w (unitless): softness indicator of fluid filled crack, w=Kfl/alpha_/K0, soft fluid saturation is w is the order of 1 

    Returns:
        eqs to be solved
    
    """    
    nu_eff, D = params
    
    eq1 = 45/16 * (nu_0-nu_eff)/(1-nu_eff**2)*(2-nu_eff)/(D*(1+3*nu_0)*(2-nu_eff)-2*(1-2*nu_0)) - crd   # eq 23 in OC&R, 1974
    eq2 = crd * D**2-( crd+9/16 * (1-2*nu_eff)/(1-nu_0**2) + 3*w/(4*np.pi))*D + 9/16 * (1-2*nu_eff)/(1-nu_0**2) # eq 23 in OC&R, 1974
    return  [eq1,eq2]




In [23]:
def test_func(params, X,K,G):
    """_summary_

    Args:
        params (_type_): _description_
        X (): _description_
        K (_type_): _description_
        G (_type_): _description_
    """    

    x, y = params
    
    eq1 = np.dot(X, (K-x))*x*y
    eq2 = np.dot(X, (G-y))*2*x*y
    return  [eq1,eq2]


In [52]:
X = np.array([0.7,0.29,0.01])#.reshape((3,1))
K = np.array([37,2.25,2.25])#.reshape((3,1))
G = np.array([44,0,0])#.reshape((3,1))
Alpha = np.array([1,1,0.01])
#The Voigt average may be taken as the starting point.
x,y=  fsolve(Berrymann_func, (K.mean(), G.mean()), args = (X,K,G,Alpha))  

In [55]:
Berrymann_sc(K,G,X,Alpha)

(16.799373768236396, 11.56317160161939)

In [57]:
if isinstance(X, ( np.ndarray)) is False:
    print('wrong')
else: 
    print('true')

true


In [37]:
alpha= np.array([1.1,0.1,0.01])#3.reshape((3,1))
P, Q= PQ(37,46,K,G, alpha)
P,Q.shape

(array([1.26113902, 1.47800095, 1.49559678]), (3,))

In [40]:
alpha= np.array([1,0.1,0.01])#3.reshape((3,1))
P, Q= PQ(37,46,K,G, alpha)
P,Q.shape

(array([1.34090916, 1.47800095, 1.49559678]), (3,))

In [14]:
X.shape

(3, 1)

In [19]:
np.dot(X,K)

17.2

In [7]:
(1+4/(3*np.pi) *(1-0.25)/(1-2*0.23) * 24/15 * 0.01)**-1

0.9906567162489948

In [9]:
def func(x,a):
  return [x[0]+x[1]-a, 3*x[0]+7*x[1]-10]
def solve(var, a):
  sol = fsolve(func, x0=var, args=(a))
  return sol

a = np.array([[1,2], [3,4]])
vfunc = np.vectorize(solve, excluded=['var'], otypes=[list])
sol = vfunc(var=np.zeros(2), a=a)

In [10]:
sol

array([[array([-0.75,  1.75]), array([1., 1.])],
       [array([2.75, 0.25]), array([ 4.5, -0.5])]], dtype=object)

monte carlo

https://people.duke.edu/~ccc14/sta-663/MonteCarlo.html

In [4]:
crd= 0.001
alpha=1
K=37
G=46
Ki=Gi=0
lamda=20
theta=30
theta= np.deg2rad(theta) 
lamda = K-2*G/3 
kapa = (Ki+4*Gi/3)*(lamda+2*G)/(np.pi*alpha*G*(lamda+G))
M = 4*Gi*(lamda+G)/(np.pi*alpha*G*(3*lamda+4*G))
U1 = 16*(lamda+2*G)/(3*(3*lamda+4*G)*(1+M))
U3 = 4*(lamda+2*G)/(3*(lamda+G)*(1+kapa))

In [5]:
C12_1 = -crd/(2*G)*(U3*(2*lamda**2+4*lamda*G*np.sin(theta)**2+G**2*np.sin(theta)**4)-U1*G**2*np.sin(theta)**4)

C11_1 = -crd/(2*G)*(U3*(2*lamda**2+4*lamda*G*np.sin(theta)**2+3*G**2*np.sin(theta)**4)+U1*G**2*np.sin(theta)**2*(4-3*np.sin(theta)**2))

C66_1 = -crd/2*G*(U3*4*np.sin(theta)**4+U1*np.sin(theta)**2*(2-np.sin(theta)**2))

In [6]:
C12_1, C11_1,C66_1,C11_1-2*C66_1

(-0.010005696588062246,
 -0.06920079383773889,
 -0.04040168875222684,
 0.011602583666714794)

In [3]:
VRH(np.array([0.2,0.5]),np.array([35,50]))

(32.0, 63.63636363636363, 47.81818181818181)

In [ ]:
def Eshelby_Cheng(K, G, phi, alpha, Kf, mat=False):
    """ Compute the effective anisotropic moduli of a cracked isotropic rock with single set fracture using Eshelby–Cheng Model.

    Args:
        K (GPa): bulk modulus of the isotropic matrix
        G (GPa): shear modulus of the isotropic matrix
        phi (frac): (crack) porosity
        alpha (unitless): aspect ratio of crack
        Kf (GPa):  bulk modulus of the fluid
        mat (bool, optional): If true: the output is in matrix form, otherwise  is numpy array. Defaults to False.

    Returns:
        C_eff: effective moduli of cracked, transversely isotropic rocks
    Refs:
        section 4.14 in The Rock Physics Handbook 
    Written by Jiaxin Yu 
    """    

    lamda= K-2*G/3 

    sigma = (3*K-2*G)/(6*K+2*G)
    R = (1-2*sigma)/(8*np.pi*(1-sigma))
    Q = 3*R/(1-2*sigma)
    Sa = np.sqrt(1-alpha**2)
    Ia = 2*np.pi*alpha*(np.arccos(alpha)-alpha*Sa)/Sa**3
    Ic = 4*np.pi-2*Ia
    Iac = (Ic-Ia)/(3*Sa**2)
    Iaa = np.pi-3*Iac/4
    Iab = Iaa/3

    S11 = Q*Iab+R*Ia
    S33 = Q*(4*np.pi/3 - 2*Iac*alpha**2)+Ic*R
    S12 = Q*Iab-R*Ia
    S13 = Q*Iac*alpha**2-R*Ia
    S31 = Q*Iac-R*Ic 
    S1212 = Q*Iab+R*Ia
    S1313 = Q*(1+alpha**2)*Iac/2 + R*(Ia+Ic)/2

    C = Kf/( 3*(K-Kf))
    D = S33*S11+S33*S12-2*S31*S13-(S11+S12+S33-1-3*C) - C*(S11+S12+2*(S33-S13-S31))
    E = S33*S11 - S31*S13 - (S33+S11-2*C-1) + C*(S31+S13-S11-S33)

    C11 = lamda*(S31-S33+1) + 2*G*E/ (D*(S12-S11+1))
    C33 = ((lamda+2*G)*(-S12-S11+1)+2*lamda*S13+4*G*C)/D
    C13 = ((lamda+2*G)*(S13+S31)-4*G*C+lamda*(S13-S12-S11-S33+2))/(2*D)
    C44 = G/(1-2*S1313)
    C66 = G/(1-2*S1212)

    if mat==True:
        C_eff = np.array([C11,C33,C13,C44,C66])
    else: 
        C_eff = write_VTI_matrix(C11,C33,C13,C44,C66)
    return C_eff

In [3]:
a= [1,2,3]
type(a)

list